In [1]:
# Utiles
    # Lib: https://github.com/sammchardy/python-binance
    # Binance doc : https://github.com/binance-exchange/binance-official-api-docs/blob/master/rest-api.md
    
from binance.client import Client
api_key = '8te7KGDgi6ayc8slY46KbWztB7yuIZ533fCrLWeqARAxBKAuQiMyyfIM4mfDrSlk'
api_secret = 'ykwjZzkYF4Bvh0T7L1OokHfHTnVjKJ4OIvaEXkV97NisgI4JwDW7jPd3pzeVVpHh'

client = Client(api_key, api_secret)

In [22]:
# ============= Check before trading =============

# For global API system:  msg: 'normal', status: 0
client.get_system_status()

# For a trading pair status = 'TRADING'
client.get_symbol_info('BNBBTC')

# Get server time
client.get_server_time()

# Exchange info to verify tradingpair exists
client.get_exchange_info()

# msg: 'Normal', success: True
client.get_account_status()

{'msg': 'Normal', 'success': True}

In [26]:
# get all symbol prices
prices = client.get_all_tickers()

# get market depth (see what should be approx the price, use to determine limit to put to be sure to buy / sell directly)
depth = client.get_order_book(symbol='ETHUSDT')

In [29]:
# canTrade=True
info = client.get_account()

# balance for an asset
client.get_asset_balance(asset='VET')

{'asset': 'VET', 'free': '70648.90000000', 'locked': '0.00000000'}

In [36]:
# current open orders (Symbol optional)
client.get_open_orders(symbol='ETHUSDT')

[]

In [31]:
# place a test market buy order, to place an actual order use the create_order function
order = client.create_test_order(
    symbol='BNBBTC',
    side=Client.SIDE_BUY,
    type=Client.ORDER_TYPE_MARKET,
    quantity=1000) # real: create_test_order